# Webスクレイピング
ここでは例として、「国土交通白書」「環境白書」から文書を抜き出し、利用します。<br>
省庁系のHP上の情報は、ほとんどの場合、出展や加工方法を明記することで、複製や公衆送信など、自由に利用ができます。
（詳細は各省庁サイトの「著作権について」を参照）

In [1]:
## 必要なライブラリの読み込み
import glob
import requests
import re
from bs4 import BeautifulSoup  ### <- anaconda 3系の最新バージョンでは最初から利用可能な状態。但し、無い場合はインストールが必要。

### 1.国土交通白書
出展: 国土交通省ホームページ 国土交通白書  URL： http://www.mlit.go.jp/statistics/file000004.html<br>
H28-H30年度分のテキスト文書データをダウンロードし格納します。<br>
各年度の白書は、左のメニューで章をクリックすると、右のウィンドウに本文が表示されます。左のメニューからリンクを辿って、右下のテキスト形式のデータを取得し、文字コード変換をして連結しています。

In [2]:
for y in range(28,31):
    urlbase = "http://www.mlit.go.jp/hakusyo/mlit/h" + str(y - 1) + "/hakusho/h" + str(y)
    req1 = requests.get( urlbase + "/menu.html" )
    with open("mlit_y" + str(y) + ".txt",mode='wb') as f:
        res = re.finditer('<a\s+href=\"html/(\w+)\.html',req1.content.decode(req1.encoding))
        for m in res:
            req2 = requests.get( urlbase + "/text/" + m.group(1) + ".txt" )
            f.write(req2.content.decode(encoding='cp932').encode('utf-8'))
glob.glob('mlit_y??.txt')  ## ファイルが出来たことを確認

['mlit_y30.txt', 'mlit_y28.txt', 'mlit_y29.txt']

### 2.環境白書
出展: 環境省ホームページ 過去の環境白書  URL: http://www.env.go.jp/policy/hakusyo/past_index.html<br>
H28-H30年度分のhtml文書中から文書部分のデータを格納します。<br>
各年度の白書は、目次ページから章や節をクリックすると、各章・節が表示されます。目次からリンクを辿り、各パートのhtml文書からタグを全て除去し（BeautifulSoup使用）、残ったテキスト部分を連結して保管します。リンクにはアンカー（#～～）が使われていて、同じhtmlファイルへのリンクが複数出てくることになる為、重複して同じhtmlを取得しないよう調整しています。

In [3]:
f_done = {}
for y in range(28,31):
    urlbase = "http://www.env.go.jp/policy/hakusyo/h" + str(y)
    req1 = requests.get( urlbase + "/index.html" )
    with open("env_y" + str(y) + ".txt",mode='w',encoding='utf-8') as f:
        res = re.finditer('<a\s+href=\"(html/\w+\.html)',req1.content.decode(req1.encoding))
        for m in res:
            if m.group(1) not in f_done:
                req2 = requests.get( urlbase + "/" + m.group(1) )
                soup = BeautifulSoup(req2.content,'html.parser',from_encoding='utf-8')
                f.write(soup.get_text())
                f_done[m.group(1)] = 1
glob.glob('env_y??.txt')   ## ファイルが出来たことを確認

['env_y30.txt', 'env_y29.txt', 'env_y28.txt']

### うまくいかない場合
`（ここはうまくいっていれば実行不要です）`<br>
ここまでうまくいっていれば、<br>
mlit_y28.txt ～ mlit_y30.txt および env_y28.txt ～ env_y30.txt <br>
が作成されているはずです。開いて確認してみてください。<br>
もしネットワーク接続がうまくいかない等の場合、内容が空で作成されてしまいます。<br>
その場合は、下記のセクションを実行して下さい。
同梱してある既にダウンロード済みのファイルを使います。

In [ ]:
import shutil,glob
for f in glob.glob('file/mlit_y??.txt'):
    shutil.copy(f,'./')
for f in glob.glob('file/env_y??.txt'):
    shutil.copy(f,'./')
[glob.glob('./mlit_y??.txt'),glob.glob('./env_y??.txt')]  ## ファイルが出来たことを確認